### Relacionando bibliotecas necessárias para o processamento

In [1]:
from datetime import datetime, date, timedelta
import datetime
import os
import time
import pandas as pd
import regex as re
import pyautogui
import zipfile
import locale
import urllib.request
import PySimpleGUI as sg
from threading import Thread
import sys
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
import pdfplumber
import easygui
from openpyxl import Workbook, load_workbook
from openpyxl.workbook import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border, Color, Alignment
from openpyxl.formula.translate import Translator
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from datetime import datetime
import openpyxl
import xlwings as xw
import numpy as np
from workadays import workdays as wd
import win32com.client as w3c
from threading import Thread
import warnings
import threading, queue
from PIL import Image, ImageTk, ImageSequence
import xlrd
import warnings
warnings.filterwarnings("ignore")

### Lendo diretório onde estão localizados os arquivos de notas fiscais

In [16]:
diretorio = os.getcwd()

diretorio_resultado = diretorio + '\Notas\Base Clientes Midori.xlsx'

Lista_Arquivos = os.listdir(diretorio + '\\XML')

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

Base_XML = pd.DataFrame()
Base_Full = pd.DataFrame()

linha=0

Base_Full['Chave_NFE'] = ''
Base_Full['Cod_NFE'] = ''
Base_Full['Data_Emissao'] = ''
Base_Full['CPF/CNPJ'] = ''
Base_Full['Nome'] = ''
Base_Full['Bairro'] = ''
Base_Full['Municipio'] = ''
Base_Full['Estado'] = ''
Base_Full['CEP'] = ''
Base_Full['Telefone'] = ''
Base_Full['Total_Pago'] = ''

for Cod_XML in Lista_Arquivos:
    
    if linha <= len(Lista_Arquivos):
        arquivo = diretorio + '\\XML\\'+Cod_XML
        Base_XML_NFE = pd.read_xml(arquivo,xpath="//doc:ide",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Dest = pd.read_xml(arquivo,xpath="//doc:dest",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_EnderDest = pd.read_xml(arquivo,xpath="//doc:enderDest",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Produto = pd.read_xml(arquivo,xpath="//doc:prod",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Pagamento = pd.read_xml(arquivo,xpath="//doc:detPag",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        Base_XML_Chave_NFE = pd.read_xml(arquivo,xpath="//doc:infNFe",namespaces = {"doc": "http://www.portalfiscal.inf.br/nfe"})
        
        # Verifica existencia de dados de CPF ou CNPJ
        if 'CPF' in Base_XML_Dest.columns :
            
            Base_XML_EnderDest['CPF/CNPJ'] = Base_XML_Dest['CPF']
            Base_XML_Produto['CPF/CNPJ'] = Base_XML_Dest['CPF']
        else:
            Base_XML_EnderDest['CPF/CNPJ'] = Base_XML_Dest['CNPJ']
            Base_XML_Produto['CPF/CNPJ'] = Base_XML_Dest['CNPJ']
        
        # Verifica existencia de dados de telefone
        if 'fone' in Base_XML_EnderDest.columns :
            
            Base_XML_EnderDest['fone'] = Base_XML_EnderDest['fone']
            
        else:
            Base_XML_EnderDest['fone'] = ''
        
        
        data=str(Base_XML_NFE['dhEmi'])
        data_emissao = data[1:15]
        
        Base_Full = Base_Full.reset_index(drop=True)

        Base_Full.loc[linha,'Chave_NFE'] = Base_XML_Chave_NFE.loc[0,'Id']
        Base_Full.loc[linha,'Cod_NFE'] = Base_XML_NFE.loc[0,'nNF']
        Base_Full.loc[linha,'Data_Emissao'] = data_emissao
        Base_Full.loc[linha,'CPF/CNPJ'] = Base_XML_EnderDest.loc[0,'CPF/CNPJ']
        Base_Full.loc[linha,'Nome'] = Base_XML_Dest.loc[0,'xNome']
        Base_Full.loc[linha,'Bairro'] = Base_XML_EnderDest.loc[0,'xBairro']
        Base_Full.loc[linha,'Municipio'] = Base_XML_EnderDest.loc[0,'xMun']
        Base_Full.loc[linha,'Estado'] = Base_XML_EnderDest.loc[0,'UF']
        Base_Full.loc[linha,'CEP'] = Base_XML_EnderDest.loc[0,'CEP']
        Base_Full.loc[linha,'Telefone'] = Base_XML_EnderDest.loc[0,'fone']
        Base_Full.loc[linha,'Total_Pago'] = Base_XML_Pagamento.loc[0,'vPag']
        
        linha=linha+1
    
with pd.ExcelWriter(diretorio_resultado) as escrever:
    Base_Full.to_excel(escrever, sheet_name ='Base_Clientes') 
    
Base_Full.head()

,Chave_NFE,Cod_NFE,Data_Emissao,CPF/CNPJ,Nome,Bairro,Municipio,Estado,CEP,Telefone,Total_Pago
0,NFe33201042498675000152558900030674911881408774,3067491,2020-10-14,14120132757,Julio Lamblet Costa e Sousa,Santos Dumont,São José do Rio Preto,SP,15020040,17981023429,149.90
1,NFe33201042498675000152558900030749221895742965,3074922,2020-10-16,17578752200,Ivan De Araujo,Quarenta Horas (Coqueiro),Ananindeua,PA,67120370,,149.90
2,NFe33201042498675000152558900030793131000677741,3079313,2020-10-19,11777040990,Andrey Turatto Grelak,Parque São Paulo,Cascavel,PR,85802030,45998192747,109.90
3,NFe33201042498675000152558900030793311355273972,3079331,2020-10-19,1252608497,Rogério Castro Guimarães,Planalto,Natal,RN,59073120,8499600566,129.90
4,NFe33201042498675000152558900030865181246369021,3086518,2020-10-21,6560313107,Mariana Da Silva Santos,Sobradinho,Brasília,DF,73005108,61996901243,184.90
